# Calculate Embeddings

Ajuda a calcular embeddings dos dados adquiridos dos comentários das redes sociais de políticos.

Entrada: `LulaTotal Validado.xlsx` e `Bolsonaro Validado.xlsx`

Saída: `Embeddings_(NOME_DO_MODELO)_(REDE_SOCIAL).xlsx`

In [964]:
import pandas as pd
import re
from sentence_transformers import SentenceTransformer

BASE_PATH = 'dados/'
DIRECTORY = BASE_PATH + 'preprocessed/embeddings/'
SOCIAL_NETWORK = 'tiktok'

In [965]:
column_types = {'ID' : str}

Iniciação das variáveis que iremos utilizar e das colunas na qual iremos trabalhar


In [966]:
# 1. Pegar o modelo para testar
TYPE_MODEL = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
model = SentenceTransformer(TYPE_MODEL)

# 2. Pegar as sentenças (nesse caso, no Post-filtrado)
file_pathL = BASE_PATH + 'input/LulaTotal Validado.xlsx'
file_pathB = BASE_PATH + 'input/Bolsonaro Validado.xlsx'
file_path_features = 'Embeddings_paraphrase-multilingual-MiniLM_'+(SOCIAL_NETWORK) +'.xlsx'
#é bom modificar o nome manualmente, pois pode dar erro na questão do save

column_text = "Texto"
column_id = "ID"
column_author = "Perfil"
column_likes = "Curtidas"
column_inicial_date = "DataColeta"
column_final_date = "DataPost"
column_cand = "Candidato"

In [967]:
# converte para datetime 
def adjust_dates(df_aux):
    df_aux['DataColeta'] = pd.to_datetime(df_aux['DataColeta'])
    df_aux['DataPost'] = pd.to_datetime(df_aux['DataPost'])

    try:
        # converte esta coluna para conter uma string com o dia da semana
        df_aux['DiaDaSemana'] = pd.to_datetime(df_aux['DiaDaSemana'])
        df_aux['DiaDaSemana'] = df_aux['DiaDaSemana'].dt.strftime('%A')
    except:
        print("Coluna DiaDaSemana parece já estar guardado como date. Não precisa transformar!")

In [968]:
# 3. Ler os Arquivos e Remover NaN
rfL = pd.read_excel(file_pathL, dtype=column_types)
rfB = pd.read_excel(file_pathB, dtype=column_types)

rfL.shape, rfB.shape


((308, 22), (269, 22))

In [969]:
rf_total = pd.concat([rfL, rfB], axis=0)
rf_total.shape

(577, 22)

In [970]:
len(rf_total['ID'].unique())


577

In [971]:
rf_total = rf_total.reset_index(drop=True)
rf_total.head(5)

,Unnamed: 0,DataColeta,Perfil,DataPost,DiaDaSemana,Plays,Curtidas,Comentarios,Compart.,Texto,...,LinkPost,ID,Duracao,Retórica Aristotélica,Dispositivo Retórico,Tipo de conteúdo,Abordagem,Tonalidade,Main character,Texto / Hashtag
0,1,2022-10-02,lulaoficial,2022-06-30 00:00:00,1900-01-05 00:00:00,196800.0,11700,809,589.0,Alô alô geração tiktoker! Imagina só um #gover...,...,https://www.tiktok.com/@lulaoficial/video/7115...,7115033431473474822,17.30,Pathos,Political Statement,Political-Purposeful,Acclamation,Neutral,Self alone,Texto + Hashtag
1,2,2022-10-02,lulaoficial,2022-06-30 00:00:00,1900-01-05 00:00:00,522000.0,33600,3324,3973.0,Já imaginou um #governo feito pra que as pesso...,...,https://www.tiktok.com/@lulaoficial/video/7115...,7115174031162215686,60.16,Pathos,Political Statement,Political-Purposeful,Acclamation,Neutral,Self alone,Texto + Hashtag
2,3,2022-10-02,lulaoficial,2022-07-01 00:00:00,1900-01-06 00:00:00,427900.0,34600,2289,1752.0,O pai ta estourado!😎 marque nos comentários um...,...,https://www.tiktok.com/@lulaoficial/video/7115...,7115357413712153861,14.88,Ethos,Humor,Campaign Act,Acclamation,Positive,Self alone,Texto + Hashtag
3,4,2022-10-02,lulaoficial,2022-07-01 00:00:00,1900-01-06 00:00:00,882200.0,47500,4312,2257.0,A gente tem um um encontro marcado no dia 02 d...,...,https://www.tiktok.com/@lulaoficial/video/7115...,7115560675824422149,15.39,Pathos,Call to Action,Campaign Act,Acclamation,Positive,Self alone,Texto + Hashtag
4,5,2022-10-02,lulaoficial,2022-07-02 00:00:00,1900-01-07 00:00:00,262200.0,22400,2150,1438.0,#PepsiApplePieChallenge Estamos fazendo uma #c...,...,https://www.tiktok.com/@lulaoficial/video/7115...,7115793869152734470,60.93,Pathos,Commitment,Political-Ideological,Acclamation,Positive,Self + voters,Texto + Hashtag


In [972]:
rf_total.dropna(inplace=True)

In [973]:
rf_total.shape

(568, 22)

Função para verificar se o texto contém apenas hashtags

Caso o comentário só possua hashtags ele irá retornar  `true`, caso contrário a função retorna `false`

In [974]:
rf_total[rf_total.isna().sum(axis=1) > 0].groupby('Perfil').count()

,Unnamed: 0,DataColeta,DataPost,DiaDaSemana,Plays,Curtidas,Comentarios,Compart.,Texto,LinkFoto,...,LinkPost,ID,Duracao,Retórica Aristotélica,Dispositivo Retórico,Tipo de conteúdo,Abordagem,Tonalidade,Main character,Texto / Hashtag
Perfil,,,,,,,,,,,,,,,,,,,,,


In [975]:
rf_total.loc[rf_total.isna().sum(axis=1) > 0].shape
rf_total.shape

(568, 22)

In [976]:
rf_total.drop(columns=["Unnamed: 0","Plays","Comentarios","Compart.","Duracao", "LinkFoto", "LinkVideo", "LinkPost", 
                       "Retórica Aristotélica", "Dispositivo Retórico", "Tipo de conteúdo",
                  "Abordagem", "Tonalidade", "Main character", "Texto / Hashtag"], inplace=True)
rf_total.tail(5)

,DataColeta,Perfil,DataPost,DiaDaSemana,Curtidas,Texto,ID
572,2022-10-30,bolsonaromessiasjair,2022-10-29 09:14:06.702000,1900-01-07 00:00:00,95300,#bolsonaro #rock #brasil #good #vibes #jairbol...,7159766081593150726
573,2022-10-30,bolsonaromessiasjair,2022-10-29 09:14:01.295000,1900-01-07 00:00:00,240900,- Muito brigado a você que nos acompanhou até ...,7159773640030997766
574,2022-10-30,bolsonaromessiasjair,2022-10-29 09:13:55.570000,1900-01-07 00:00:00,109600,#bolsonaro #lula #empolgado #brasil #🇧🇷,7159891102143630597
575,2022-10-30,bolsonaromessiasjair,2022-10-29 12:13:51.619000,1900-01-07 00:00:00,52100,#vacina #presidente #jair #bolsonaro #comparti...,7159943895445441797
576,2022-10-30,bolsonaromessiasjair,2022-10-29 17:13:45.948000,1900-01-07 00:00:00,23500,- Belo Horizonte/MG. - Presidente Jair Bolsona...,7160017587160485125


In [977]:
rf_total.loc[rf_total.isna().sum(axis=1) > 0].shape
rf_total.dropna(inplace=True)
rf_total.shape

(568, 7)

In [978]:
# converte para datetime
adjust_dates(rf_total)
rf_total.tail(20)

,DataColeta,Perfil,DataPost,DiaDaSemana,Curtidas,Texto,ID
557,2022-10-30,bolsonaromessiasjair,2022-10-23 09:15:17.444,Monday,837500,- 2️⃣2️⃣✅ 🇧🇷🇧🇷🇧🇷🇧🇷 #brasil #22 #é #bolsonaro #...,7157407242067660038
558,2022-10-30,bolsonaromessiasjair,2022-10-23 09:15:12.715,Monday,175400,#neymar #bolsonaro #brasil #campeao #live #deu...,7157441811538038022
559,2022-10-30,bolsonaromessiasjair,2022-10-24 09:15:07.986,Tuesday,143700,#humor #bom #dia #brasil #jair #bolsonaro #🇧🇷,7157697379070676230
560,2022-10-30,bolsonaromessiasjair,2022-10-24 09:15:02.435,Tuesday,28400,#gusttavo #lima #gusttavolima #bolsonaro #jair...,7157773223302679814
561,2022-10-30,bolsonaromessiasjair,2022-10-24 09:14:57.974,Tuesday,38700,- Apresento-lhes um socialista que defende a l...,7157899882932571398
562,2022-10-30,bolsonaromessiasjair,2022-10-25 09:14:53.942,Wednesday,34900,#jair #pai #mae #bolsonaro #mensagem #comparti...,7158062596438969605
563,2022-10-30,bolsonaromessiasjair,2022-10-26 09:14:50.342,Thursday,759100,#jair #bolsonaro #jairbolsonaro #presidente #k...,7158480559654636805
564,2022-10-30,bolsonaromessiasjair,2022-10-26 09:14:45.616,Thursday,15900,#presidente #bolsonaro,7158547810244332806
565,2022-10-30,bolsonaromessiasjair,2022-10-26 09:14:40.683,Thursday,20800,#presidente #bolsonaro #bahia #barreiras #bras...,7158599839259856134
566,2022-10-30,bolsonaromessiasjair,2022-10-26 09:14:35.278,Thursday,52500,#presidente #jair #bolsonaro #liberdade #brasi...,7158760403244256517


In [979]:
rf_total.groupby(['DataColeta']).count()

,Perfil,DataPost,DiaDaSemana,Curtidas,Texto,ID
DataColeta,,,,,,
2022-10-02,425,425,425,425,425,425
2022-10-30,143,143,143,143,143,143


In [980]:
rf_total['DiasDecorridos'] = (rf_total['DataColeta'] - rf_total['DataPost']).dt.days
rf_total

rf_total['Candidato'] = rf_total['Perfil'].apply(lambda x: 'Lula' if x == 'lulaoficial' else 'Bolsonaro')

In [981]:
import re

def contains_only_hashtags(text):
    hashtags = re.findall(r'#\S+', text)
    return len(hashtags) == len(text.split())

In [982]:
ids = rf_total[column_id].tolist()
authors = rf_total[column_author].tolist()
sentences = rf_total[column_text].tolist()
likes = rf_total[column_likes].tolist()
days = rf_total['DiasDecorridos'].tolist()
candidatos = rf_total['Candidato'].tolist()

Encode dos embeddings utilizando a library do "sentence-transformers" e também a concatenação de dados previamente informados da planilha passada como `ID`,`Candidato` e `Curtidas`

In [983]:
# 4. Calcular os embeddings das sentenças
embeddings = model.encode(sentences)
df_embeddings = pd.DataFrame(embeddings) 
df_embeddings.columns = [f'x{i+1}' for i in range(df_embeddings.shape[1])]



In [984]:
df_embeddings.tail(10)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x375,x376,x377,x378,x379,x380,x381,x382,x383,x384
558,0.065774,0.076513,-0.005098,-0.105385,0.368520,-0.009180,0.144425,0.209637,0.160555,0.180756,...,0.140331,-0.286585,-0.186765,-0.032015,0.039835,0.085074,0.305739,-0.098804,-0.037546,0.074582
559,0.018181,0.036821,0.082955,-0.052921,0.392086,0.095935,0.080529,0.193297,0.057481,0.194274,...,0.285527,-0.174320,-0.221916,-0.055774,0.064263,0.181429,0.293877,-0.080864,-0.214063,0.159148
560,0.101947,0.060581,0.047223,-0.031386,0.136382,-0.085900,0.440422,0.207936,0.071522,0.087409,...,0.019699,-0.376846,-0.132506,0.139560,0.049472,0.120462,0.479986,0.041716,-0.236809,0.178856
561,0.014851,0.022353,-0.054209,-0.021883,0.090301,-0.028910,0.153615,0.285314,-0.046663,0.062689,...,0.015151,-0.143126,-0.231852,-0.167745,0.128587,0.064740,0.241012,-0.170597,-0.182016,0.118491
562,0.061470,0.272903,-0.007901,-0.095093,0.077055,0.121772,0.104048,0.053682,0.013486,0.047381,...,0.403106,0.005631,-0.054613,-0.036821,-0.157545,0.115093,0.025409,0.170641,-0.223352,0.017441
563,0.062468,-0.165707,-0.014988,-0.046828,-0.138289,0.048322,0.310527,0.314963,-0.100900,-0.065606,...,-0.002563,-0.106669,-0.268587,0.030322,-0.096264,-0.001911,0.384258,-0.307925,-0.035968,0.234999
564,-0.163468,0.077066,-0.156664,-0.085776,0.133453,0.225597,0.063915,-0.064023,0.044049,0.114685,...,0.317411,-0.004120,0.018524,-0.022853,-0.236356,0.061097,0.077973,0.205867,0.109763,0.176917
565,0.131149,-0.077428,-0.067303,-0.020109,0.218621,-0.061110,0.275060,0.250501,0.057850,0.129988,...,0.062790,-0.291914,-0.257054,-0.115412,0.018693,0.142378,0.403265,-0.229022,-0.246011,0.117278
566,0.011311,-0.063400,0.107073,-0.160402,0.371433,0.144655,0.102610,0.245211,0.122302,0.229537,...,0.262899,-0.199217,-0.148512,-0.104007,0.125126,0.068031,0.212707,-0.136398,-0.033402,0.186931
567,0.018853,0.130617,-0.012598,-0.090308,0.292937,0.103336,-0.023820,0.119030,0.077762,0.114861,...,0.281803,-0.258964,-0.153638,-0.012124,-0.048347,0.000193,0.230905,0.010980,-0.234178,0.090194


A planilha ficará com as colunas importantes originais (__ID__,__Candidato__ e __Curtidas__), e também estará com uma flag que mostra se há hashtags(__Only Hashtags__), e por fim, features calculadas pelo embeddinG   __x1__,__x2__,__x3__ ... __xN__ , onde N seria o tamanho de dimensões que aquele modelo possui

# Concatenar e Salvar o Arquivo

In [985]:
df_final = pd.DataFrame({
    column_id: ids,
    'Candidato': candidatos,
    column_likes: likes,
    'Dias Decorridos': days
    
})

df_final['Only Hashtags'] = rf_total[column_text].apply(contains_only_hashtags)
df_final['Only Hashtags'] = df_final['Only Hashtags'].fillna(False)  # Para lidar com tabulações que o pandas não reconhece

# Concatena com df_embeddings
df_final = pd.concat([df_final, df_embeddings], axis=1)


C:\Users\Kabum\AppData\Local\Temp\ipykernel_18116\2317160244.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final['Only Hashtags'] = df_final['Only Hashtags'].fillna(False)  # Para lidar com tabulações que o pandas não reconhece


In [986]:
df_final.tail(10)


,ID,Candidato,Curtidas,Dias Decorridos,Only Hashtags,x1,x2,x3,x4,x5,...,x375,x376,x377,x378,x379,x380,x381,x382,x383,x384
558,7159137058555022598,Bolsonaro,97200,2,True,0.065774,0.076513,-0.005098,-0.105385,0.368520,...,0.140331,-0.286585,-0.186765,-0.032015,0.039835,0.085074,0.305739,-0.098804,-0.037546,0.074582
559,7159293944143138053,Bolsonaro,48900,1,True,0.018181,0.036821,0.082955,-0.052921,0.392086,...,0.285527,-0.174320,-0.221916,-0.055774,0.064263,0.181429,0.293877,-0.080864,-0.214063,0.159148
560,7159536039252577542,Bolsonaro,401100,0,True,0.101947,0.060581,0.047223,-0.031386,0.136382,...,0.019699,-0.376846,-0.132506,0.139560,0.049472,0.120462,0.479986,0.041716,-0.236809,0.178856
561,7159681566732520710,Bolsonaro,110500,0,False,0.014851,0.022353,-0.054209,-0.021883,0.090301,...,0.015151,-0.143126,-0.231852,-0.167745,0.128587,0.064740,0.241012,-0.170597,-0.182016,0.118491
562,7159695635761876230,Bolsonaro,61300,0,True,0.061470,0.272903,-0.007901,-0.095093,0.077055,...,0.403106,0.005631,-0.054613,-0.036821,-0.157545,0.115093,0.025409,0.170641,-0.223352,0.017441
563,7159766081593150726,Bolsonaro,95300,0,True,0.062468,-0.165707,-0.014988,-0.046828,-0.138289,...,-0.002563,-0.106669,-0.268587,0.030322,-0.096264,-0.001911,0.384258,-0.307925,-0.035968,0.234999
564,7159773640030997766,Bolsonaro,240900,0,True,-0.163468,0.077066,-0.156664,-0.085776,0.133453,...,0.317411,-0.004120,0.018524,-0.022853,-0.236356,0.061097,0.077973,0.205867,0.109763,0.176917
565,7159891102143630597,Bolsonaro,109600,0,True,0.131149,-0.077428,-0.067303,-0.020109,0.218621,...,0.062790,-0.291914,-0.257054,-0.115412,0.018693,0.142378,0.403265,-0.229022,-0.246011,0.117278
566,7159943895445441797,Bolsonaro,52100,0,True,0.011311,-0.063400,0.107073,-0.160402,0.371433,...,0.262899,-0.199217,-0.148512,-0.104007,0.125126,0.068031,0.212707,-0.136398,-0.033402,0.186931
567,7160017587160485125,Bolsonaro,23500,0,True,0.018853,0.130617,-0.012598,-0.090308,0.292937,...,0.281803,-0.258964,-0.153638,-0.012124,-0.048347,0.000193,0.230905,0.010980,-0.234178,0.090194


# Salvar o arquivo

In [987]:

df_final.to_excel((DIRECTORY + file_path_features), index=False)